## Packages

In [82]:
install.packages("fixest")

Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

also installing the dependencies ‘dreamerr’, ‘stringmagic’




In [8]:
install.packages("car")
install.packages("dplyr")
install.packages("haven")
install.packages("lfe")
install.packages("sandwich")
install.packages("plm")
install.packages("stargazer")
install.packages("corrplot")
install.packages("ivreg")
install.packages("gbm")
install.packages("fixest")

Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)



Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘miscTools’, ‘generics’, ‘rbibutils’, ‘bdsmatrix’, ‘collapse’, ‘lmtest’, ‘maxLik’, ‘Rdpack’




In [83]:
library(haven)
library(lfe)
library(lmtest)
library(dplyr)
library(stargazer)
library(sandwich)
library(corrplot)
library(ivreg)
library(purrr)
library(stringr)
library(tidyr)
library(xtable)
library(lfe)
library(mgcv)
library(gbm)
library(fixest)


Attaching package: ‘fixest’


The following object is masked from ‘package:lfe’:

    fepois




In [114]:
set.seed(123)
star_points = c(0, 0.01, 0.05, 0.1, 1)
star_symbols = c("***", "**", "*", " ")

wrap_with_makecell <- function(x) {
  paste0("\\makecell{", x, "}")
}

print_latex <- function (table_data) {
  table_data <- lapply(table_data, function(x) lapply(x, wrap_with_makecell))
  table_data <- data.frame(table_data, check.names = FALSE)
  table_data <- xtable(table_data)
  print(table_data, sanitize.text.function = function(x) gsub("\n", "\\\\\\\\", x))
}

In [100]:
extract_2_coefs_factory <- function(feature1, feature2) {
  extract_coefs_fn <- function(model) {
    summary_model <- summary(model)
    coefs <- summary_model$coefficients

    # Find the rows for experiment_treatment_commute120 and experiment_treatment
    treatment_row <- which(row.names(coefs) == feature1)
    announcement_experiment_row <- which(row.names(coefs) == feature2)

    # Extract coefficients and standard errors
    data.frame(
      feature = c(feature1, feature2),
      values = c(
        if (length(treatment_row) > 0) {
          stars <- symnum(coefs[treatment_row, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
          str_c(round(coefs[treatment_row, 1], digits = 4), stars, "\n(", round(coefs[treatment_row, 2], digits = 4), ")") 
        } else "",
        if (length(announcement_experiment_row) > 0) {
          stars <- symnum(coefs[announcement_experiment_row, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
          str_c(round(coefs[announcement_experiment_row, 1], digits = 4), stars, "\n(", round(coefs[announcement_experiment_row, 2], digits = 4), ")")
        } else ""
      )
    )
  }

  extract_coefs_fn
}

## Table 2

In [115]:
performance_during_exper <- haven::read_dta("data/performance_during_exper.dta")

col_1_regression <-  felm(perform1 ~ experiment_treatment + as.factor(year_week)
                          | personid | 0 | personid, 
                          data=performance_during_exper, 
                          subset=(year_week != 201049 & (expgroup == 1 | expgroup == 0)))

col_2_regression <- felm(perform1 ~ treatment + as.factor(year_week) | 0 | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week > 201049 & (expgroup == 1 | expgroup == 0)))

col_3_regression <- felm(phonecall ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logcallpersec) & phonecallraw > 20))

col_4_regression <- felm(logphonecall ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logcallpersec) & phonecallraw > 20))

col_5_regression <- felm(logcallpersec ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

col_6_regression <- felm(logcalllength ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

# wage_data <- read.dta("wage_new.dta")
# col_7_regression <- felm(loggrosswage ~ experiment_treatment + as.factor(wage_month) | personid | 0 | personid,
#                          data = wage_data,
#                          subset = (wage_month < 201109))

# extract_first_regressor <- function(model) {
#   summary_model <- summary(model)
#   coefs <- summary_model$coefficients
  
#   non_intercept_rows <- row.names(coefs) != "(Intercept)"
#   first_non_intercept <- which(non_intercept_rows)[1]
  
#   coef <- coefs[first_non_intercept, 1]
#   se <- coefs[first_non_intercept, 2]
#   stars <- symnum(coefs[first_non_intercept, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
  
#   regressor <- row.names(coefs)[first_non_intercept]
  
#   data.frame(regressor = regressor,
#              coef_se = str_c(round(coef, digits=4), stars, "\n(", round(se, digits=4), ")"))
# }

extract_first_regressor <- extract_2_coefs_factory("experiment_treatment", "treatment")

model_list <- list(col_1_regression, col_2_regression, col_3_regression, col_4_regression, col_5_regression, col_6_regression)

model_data <- model_list %>%
  map_dfr(extract_first_regressor, .id = "model")

table_data <- model_data %>%
  pivot_wider(names_from = model, values_from = values)

table_data
print_latex(table_data)

feature,1,2,3,4,5,6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
experiment_treatment,0.2324*** (0.0624),,0.2477*** (0.0577),0.1198*** (0.0248),0.0317** (0.0129),0.0881*** (0.0268)
treatment,,0.1837** (0.0853),,,,


% latex table generated in R 4.4.0 by xtable 1.8-4 package
% Thu May  9 11:53:29 2024
\begin{table}[ht]
\centering
\begin{tabular}{rllllllllllllll}
  \hline
 & feature."\\makecell{experiment_treatment}" & feature."\\makecell{treatment}" & 1."\\makecell{0.2324***\n(0.0624)}" & 1."\\makecell{}" & 2."\\makecell{}" & 2."\\makecell{0.1837**\n(0.0853)}" & 3."\\makecell{0.2477***\n(0.0577)}" & 3."\\makecell{}" & 4."\\makecell{0.1198***\n(0.0248)}" & 4."\\makecell{}" & 5."\\makecell{0.0317**\n(0.0129)}" & 5."\\makecell{}" & 6."\\makecell{0.0881***\n(0.0268)}" & 6."\\makecell{}" \\ 
  \hline
1 & \makecell{experiment_treatment} & \makecell{treatment} & \makecell{0.2324***\\(0.0624)} & \makecell{} & \makecell{} & \makecell{0.1837**\\(0.0853)} & \makecell{0.2477***\\(0.0577)} & \makecell{} & \makecell{0.1198***\\(0.0248)} & \makecell{} & \makecell{0.0317**\\(0.0129)} & \makecell{} & \makecell{0.0881***\\(0.0268)} & \makecell{} \\ 
   \hline
\end{tabular}
\end{table}


In [91]:
feols(perform1 ~ experiment_treatment + as.factor(year_week) | personid,
                   data = performance_during_exper,
                   subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0)),
                   cluster = "personid")

ERROR: Error in feols(perform1 ~ experiment_treatment + as.factor(year_week) | : Argument `subset` could not be evaluated: object 'year_week' not found


In [79]:
idx <- sample(1 : nrow(performance_during_exper), nrow(performance_during_exper) * 0.8)
performance_during_exper_train <- performance_during_exper[idx, ]
performance_during_exper_test <- performance_during_exper[-idx, ]

felm_model <- felm(perform1 ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                   data = performance_during_exper_train,
                   subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0)))

gam_model <- gam(perform1 ~ experiment_treatment + as.factor(year_week) + as.factor(personid),
                 data = performance_during_exper_train,
                 subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0)))

# AIC
felm_aic <- AIC(felm_model)
gam_aic <- gam_model$aic
cat("felm AIC:", felm_aic, "\ngam AIC:", gam_aic, "\n")

performance_during_exper_train_pred <- performance_during_exper_train[, names(performance_during_exper_train) != "personid"]
performance_during_exper_test_pred <- performance_during_exper_test[, names(performance_during_exper_test) != "personid"]

# prediction
felm_pred_train <- model.matrix(felm_model$formula, data = performance_during_exper_train_pred) %*% felm_model$coefficients
gam_pred_train <- predict(gam_model, newdata = performance_during_exper_train_pred)
felm_pred_test <- model.matrix(felm_model$formula, data = performance_during_exper_test_pred) %*% felm_model$coefficients
gam_pred_test <- predict(gam_model, newdata = performance_during_exper_test_pred)

felm_loss_train <- mean((performance_during_exper_train$perform1 - felm_pred_train)^2)
gam_loss_train <- mean((performance_during_exper_train$perform1 - gam_pred_train)^2)
felm_loss_test <- mean((performance_during_exper_train$perform1 - felm_pred_test)^2)
gam_loss_test <- mean((performance_during_exper_train$perform1 - gam_pred_test)^2)
cat("felm train loss:", felm_loss_train)
cat("gam train loss:", gam_loss_train)
cat("felm test loss:", felm_loss_test)
cat("gam test loss:", gam_loss_test)

felm AIC: 33803.46 
gam AIC: 33803.46 


Warning message in Ops.factor(experiment_treatment, as.factor(year_week)):
“‘+’ not meaningful for factors”


ERROR: Error in eval(predvars, data, env): object 'personid' not found


In [78]:
performance_during_exper_train[-personid]]

ERROR: Error in eval(expr, envir, enclos): object 'personid' not found


## Table 3

In [116]:
col_1_regression <- felm(logcalllength ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

col_2_regression <- felm(logcall_dayworked ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

col_3_regression <- felm(logdaysworked ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20 & !is.na(logcall_dayworked)))

col_4_regression <- felm(logcalllength ~ experiment_treatment_commute120 + experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

col_5_regression <- felm(logcall_dayworked ~ experiment_treatment_commute120 + experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

col_6_regression <- felm(logdaysworked ~ experiment_treatment_commute120 + experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20 & !is.na(logcall_dayworked)))

# extract_treatment120_coefs <- function(model) {
#   summary_model <- summary(model)
#   coefs <- summary_model$coefficients

#   # Find the rows for experiment_treatment_commute120 and experiment_treatment
#   treatment_row <- which(row.names(coefs) == "experiment_treatment")
#   treatment_commute120_row <- which(row.names(coefs) == "experiment_treatment_commute120")

#   # Extract coefficients and standard errors
#   data.frame(
#     feature = c("experiment_treatment", "experiment_treatment_commute120"),
#     coef_se = c(
#       if (length(treatment_row) > 0) {
#         stars <- symnum(coefs[treatment_row, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
#         str_c(round(coefs[treatment_row, 1], digits = 4), stars, "\n(", round(coefs[treatment_row, 2], digits = 4), ")") 
#       } else "",
#       if (length(treatment_commute120_row) > 0) {
#         stars <- symnum(coefs[treatment_commute120_row, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
#         str_c(round(coefs[treatment_commute120_row, 1], digits = 4), stars, "\n(", round(coefs[treatment_commute120_row, 2], digits = 4), ")")
#       } else ""
#     )
#   )
# }

extract_treatment120_coefs <- extract_2_coefs_factory("experiment_treatment", "experiment_treatment_commute120")

model_list <- list(col_1_regression, col_2_regression, col_3_regression, col_4_regression, col_5_regression, col_6_regression)

model_data <- model_list %>%
  map_dfr(extract_treatment120_coefs, .id = "model")

table_data <- model_data %>%
  pivot_wider(names_from = model, values_from = values)

table_data
print_latex(table_data)

feature,1,2,3,4,5,6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
experiment_treatment,0.0881*** (0.0268),0.0634*** (0.024),0.0247** (0.0123),0.0694** (0.0301),0.0486* (0.0272),0.0208 (0.0127)
experiment_treatment_commute120,,,,0.0693* (0.0362),0.0549* (0.0311),0.0144 (0.0167)


% latex table generated in R 4.4.0 by xtable 1.8-4 package
% Thu May  9 11:54:42 2024
\begin{table}[ht]
\centering
\begin{tabular}{rllllllllllllll}
  \hline
 & feature."\\makecell{experiment_treatment}" & feature."\\makecell{experiment_treatment_commute120}" & 1."\\makecell{0.0881***\n(0.0268)}" & 1."\\makecell{}" & 2."\\makecell{0.0634***\n(0.024)}" & 2."\\makecell{}" & 3."\\makecell{0.0247**\n(0.0123)}" & 3."\\makecell{}" & 4."\\makecell{0.0694**\n(0.0301)}" & 4."\\makecell{0.0693*\n(0.0362)}" & 5."\\makecell{0.0486*\n(0.0272)}" & 5."\\makecell{0.0549*\n(0.0311)}" & 6."\\makecell{0.0208 \n(0.0127)}" & 6."\\makecell{0.0144 \n(0.0167)}" \\ 
  \hline
1 & \makecell{experiment_treatment} & \makecell{experiment_treatment_commute120} & \makecell{0.0881***\\(0.0268)} & \makecell{} & \makecell{0.0634***\\(0.024)} & \makecell{} & \makecell{0.0247**\\(0.0123)} & \makecell{} & \makecell{0.0694**\\(0.0301)} & \makecell{0.0693*\\(0.0362)} & \makecell{0.0486*\\(0.0272)} & \makecell{0.0549*\\(0.0311

In [127]:
install.packages("reshape2")

Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

also installing the dependency ‘plyr’




In [129]:
library(reshape2)
print_latex(dcast(melt(table_data, id.vars = "feature"), variable ~ feature))

% latex table generated in R 4.4.0 by xtable 1.8-4 package
% Thu May  9 12:01:03 2024
\begin{table}[ht]
\centering
\begin{tabular}{rllllllllllllllllll}
  \hline
 & variable."\\makecell{1}" & variable."\\makecell{2}" & variable."\\makecell{3}" & variable."\\makecell{4}" & variable."\\makecell{5}" & variable."\\makecell{6}" & experiment_treatment."\\makecell{0.0881***\n(0.0268)}" & experiment_treatment."\\makecell{0.0634***\n(0.024)}" & experiment_treatment."\\makecell{0.0247**\n(0.0123)}" & experiment_treatment."\\makecell{0.0694**\n(0.0301)}" & experiment_treatment."\\makecell{0.0486*\n(0.0272)}" & experiment_treatment."\\makecell{0.0208 \n(0.0127)}" & experiment_treatment_commute120."\\makecell{}" & experiment_treatment_commute120."\\makecell{}" & experiment_treatment_commute120."\\makecell{}" & experiment_treatment_commute120."\\makecell{0.0693*\n(0.0362)}" & experiment_treatment_commute120."\\makecell{0.0549*\n(0.0311)}" & experiment_treatment_commute120."\\makecell{0.0144 \n(0.0167

In [119]:
as.data.frame(t(table_data))
print_latex(as.data.frame(t(table_data)))

,V1,V2
,<chr>,<chr>
feature,experiment_treatment,experiment_treatment_commute120
1,0.0881*** (0.0268),
2,0.0634*** (0.024),
3,0.0247** (0.0123),
4,0.0694** (0.0301),0.0693* (0.0362)
5,0.0486* (0.0272),0.0549* (0.0311)
6,0.0208 (0.0127),0.0144 (0.0167)


% latex table generated in R 4.4.0 by xtable 1.8-4 package
% Thu May  9 11:55:15 2024
\begin{table}[ht]
\centering
\begin{tabular}{rllllllllllllll}
  \hline
 & V1."\\makecell{experiment_treatment}" & V1."\\makecell{0.0881***\n(0.0268)}" & V1."\\makecell{0.0634***\n(0.024)}" & V1."\\makecell{0.0247**\n(0.0123)}" & V1."\\makecell{0.0694**\n(0.0301)}" & V1."\\makecell{0.0486*\n(0.0272)}" & V1."\\makecell{0.0208 \n(0.0127)}" & V2."\\makecell{experiment_treatment_commute120}" & V2."\\makecell{}" & V2."\\makecell{}" & V2."\\makecell{}" & V2."\\makecell{0.0693*\n(0.0362)}" & V2."\\makecell{0.0549*\n(0.0311)}" & V2."\\makecell{0.0144 \n(0.0167)}" \\ 
  \hline
1 & \makecell{experiment_treatment} & \makecell{0.0881***\\(0.0268)} & \makecell{0.0634***\\(0.024)} & \makecell{0.0247**\\(0.0123)} & \makecell{0.0694**\\(0.0301)} & \makecell{0.0486*\\(0.0272)} & \makecell{0.0208 \\(0.0127)} & \makecell{experiment_treatment_commute120} & \makecell{} & \makecell{} & \makecell{} & \makecell{0.0693*\\(0.03

## Table 4

In [9]:
performance_during_exper <- haven::read_dta("data/performance_during_exper.dta")

model_perform1_excluding_2 <- felm(
  formula = perform1 ~ experiment_treatment + experiment_control + as.factor(year_week) |
  personid | 0 | personid,
  subset = (year_week != 201049 & expgroup != 2 & !is.na(perform1)),
  data = performance_during_exper
)

# model_phonecall_excluding_2 <- felm(
#   formula = phonecall ~ experiment_treatment + experiment_control + factor(year_week) |
#   personid | 0 | personid,
#   subset = (year_week != 201049 & expgroup != 2),
#   data = performance_during_exper
# )

# model_perform1_excluding_3 <- felm(
#   formula = perform1 ~ experiment_treatment + experiment_control + factor(year_week) |
#   personid | 0 | personid,
#   subset = (year_week != 201049 & expgroup != 3),
#   data = performance_during_exper
# )

# model_phonecall_excluding_3 <- felm(
#   formula = phonecall ~ experiment_treatment + experiment_control + factor(year_week) |
#   personid | 0 | personid,
#   subset = (year_week != 201049 & expgroup != 3),
#   data = performance_during_exper
# )

summary(model_perform1_excluding_2)


Call:
   felm(formula = perform1 ~ experiment_treatment + experiment_control +      as.factor(year_week) | personid | 0 | personid, data = performance_during_exper,      subset = (year_week != 201049 & expgroup != 2 & !is.na(perform1))) 

Residuals:
    Min      1Q  Median      3Q     Max 
-4.8705 -0.4650  0.0404  0.5166  3.9519 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.19396      0.04637   4.183 3.02e-05 ***
experiment_control         -0.03467      0.04720  -0.734    0.463    
as.factor(year_week)201002  1.80792      0.04288  42.161  < 2e-16 ***
as.factor(year_week)201003  2.06448      0.04329  47.684  < 2e-16 ***
as.factor(year_week)201004  2.15205      0.04512  47.698  < 2e-16 ***
as.factor(year_week)201005  2.21217      0.04481  49.371  < 2e-16 ***
as.factor(year_week)201006  2.11384      0.04329  48.828  < 2e-16 ***
as.factor(year_week)201007  1.38370      0.04088  33.851  < 2e-16 ***
as.factor(year_week)20

## Table 5

In [10]:
selection_data <- read_dta("data/selection_new.dta")

col_1_regression <- felm(perform1 ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049 & year_week <= 201133))

col_2_regression <- felm(perform1 ~ experiment_treatment + treatment_post_experiment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049))

col_3_regression <- felm(perform1 ~ experiment_treatment + treatment_post_experiment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049 & balanced == 1))

col_4_regression <- felm(logphonecall ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049 & year_week <= 201133 & phonecallraw > 20))

col_5_regression <- felm(logphonecall ~ experiment_treatment + treatment_post_experiment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049 & phonecallraw > 20))

col_6_regression <- felm(logphonecall ~ experiment_treatment + treatment_post_experiment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049 & phonecallraw > 20 & balanced == 1))

summary(col_1_regression)


Call:
   felm(formula = perform1 ~ experiment_treatment + as.factor(year_week) |      personid | 0 | personid, data = selection_data, subset = (year_week !=      201049 & year_week <= 201133)) 

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6269 -0.4207  0.0312  0.4494  3.7859 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.24423      0.05819   4.197 3.77e-05 ***
as.factor(year_week)201002  1.63337      0.08088  20.194  < 2e-16 ***
as.factor(year_week)201003  1.76537      0.08400  21.016  < 2e-16 ***
as.factor(year_week)201004  1.87319      0.08456  22.151  < 2e-16 ***
as.factor(year_week)201005  1.72225      0.08830  19.505  < 2e-16 ***
as.factor(year_week)201006  1.71056      0.08352  20.482  < 2e-16 ***
as.factor(year_week)201007  1.48279      0.07298  20.318  < 2e-16 ***
as.factor(year_week)201008  1.68329      0.09461  17.793  < 2e-16 ***
as.factor(year_week)201009  1.57455      0.07576  20.783  < 2e-16 

In [11]:
extract_selection_coefs <- function(model) {
  summary_model <- summary(model)
  coefs <- summary_model$coefficients

  # Find the rows for experiment_treatment_commute120 and experiment_treatment
  treatment_row <- which(row.names(coefs) == "experiment_treatment")
  treatment_post_experiment_row <- which(row.names(coefs) == "treatment_post_experiment")

  # Extract coefficients and standard errors
  data.frame(
    feature = c("experiment_treatment", "treatment_post_experiment"),
    coef_se = c(
      if (length(treatment_row) > 0) {
        stars <- symnum(coefs[treatment_row, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
        str_c(round(coefs[treatment_row, 1], digits = 4), stars, "\n(", round(coefs[treatment_row, 2], digits = 4), ")") 
      } else "",
      if (length(treatment_post_experiment_row) > 0) {
        stars <- symnum(coefs[treatment_post_experiment_row, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
        str_c(round(coefs[treatment_post_experiment_row, 1], digits = 4), stars, "\n(", round(coefs[treatment_post_experiment_row, 2], digits = 4), ")")
      } else ""
    )
  )
}

model_list <- list(col_1_regression, col_2_regression, col_3_regression, col_4_regression, col_5_regression, col_6_regression)

model_data <- model_list %>%
  map_dfr(extract_selection_coefs, .id = "model")

table_data <- model_data %>%
  pivot_wider(names_from = model, values_from = coef_se)

table_data

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”


feature,1,2,3,4,5,6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
experiment_treatment,0.2442*** (0.0582),0.2212*** (0.0485),0.1739*** (0.0566),0.1116*** (0.0239),0.1017*** (0.0249),0.0981*** (0.03)
treatment_post_experiment,,0.2839*** (0.0814),0.2446*** (0.0883),,0.1046*** (0.035),0.0962** (0.0398)


## Table 7

In [12]:
satisfaction_data <- read_dta("data/satisfaction.dta")

col_1_regression = felm(log(satisfaction) ~ expgroup_treatment + factor(surveyno) | personid | 0 | personid, 
                        data = satisfaction_data)


col_2_regression = felm(log(general) ~ expgroup_treatment + factor(surveyno) | personid | 0 | personid, 
                        data = satisfaction_data)


col_3_regression = felm(log(life) ~ expgroup_treatment + factor(surveyno) | personid | 0 | personid, 
                        data = satisfaction_data)

exhaustion_data <- read_dta("data/exhaustion.dta")
col_4_regression = felm(lnexhaustion ~ experiment_expgroup + announcement_expgroup + factor(year_week) | 0 | 0 | personid, 
                        data = exhaustion_data)

positive_data <- read_dta("data/positive.dta")
col_5_regression = felm(lnpositive ~ experiment_expgroup + announcement_expgroup + factor(year_week) | 0 | 0 | personid, 
                        data = positive_data)

negative_data <- read_dta("data/negative.dta")
col_6_regression = felm(lnnegative ~ experiment_expgroup + announcement_expgroup + factor(year_week) | 0 | 0 | personid, 
                        data = negative_data)

summary(col_6_regression)


Call:
   felm(formula = lnnegative ~ experiment_expgroup + announcement_expgroup +      factor(year_week) | 0 | 0 | personid, data = negative_data) 

Residuals:
     Min       1Q   Median       3Q      Max 
-0.81996 -0.31630  0.07544  0.30110  1.07025 

Coefficients:
                        Estimate Cluster s.e. t value Pr(>|t|)    
(Intercept)              2.75030      0.04416  62.283  < 2e-16 ***
experiment_expgroup     -0.18258      0.05787  -3.155  0.00162 ** 
announcement_expgroup   -0.09495      0.05792  -1.639  0.10121    
factor(year_week)201050  0.07252      0.04299   1.687  0.09170 .  
factor(year_week)201051  0.09240      0.04731   1.953  0.05089 .  
factor(year_week)201052  0.06707      0.04642   1.445  0.14859    
factor(year_week)201053  0.11406      0.04589   2.485  0.01297 *  
factor(year_week)201102  0.06763      0.04891   1.383  0.16684    
factor(year_week)201103  0.08962      0.04895   1.831  0.06719 .  
factor(year_week)201104  0.08994      0.05264   1.709  0.0875

In [99]:
# extract_coefs_factory <- function(features) {
#   extract_coefs_fn <- function(model) {
#     summary_model <- summary(model)
#     coefs <- summary_model$coefficients

#     row_names = lapply(features, function(feature) { which(row.names(coefs) == feature) })
#     values = list()
#     for (feature in features) {
#       row_ = which(row.names(coefs) == feature)
#       if (length(row_) > 0) {
#         stars <- symnum(coefs[row_, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
#         val = str_c(round(coefs[row_, 1], digits = 4), stars, "\n(", round(coefs[row_, 2], digits = 4), ")") 
#       } else {
#         val = ""
#       }
#       values[[length(values) + 1]] = val
#     }
#     data.frame(feature = features, values = values)
#   }

#   extract_coefs_fn
# }

extract_7_1_coefs <- extract_2_coefs_factory("expgroup_treatment", "announcement_expgroup")

model_list_1 <- list(col_1_regression, col_2_regression, col_3_regression)

model_data <- model_list_1 %>%
  map_dfr(extract_7_1_coefs, .id = "model")

table_data_1 <- model_data %>%
  pivot_wider(names_from = model, values_from = values)

extract_7_2_coefs <- extract_2_coefs_factory("experiment_expgroup", "announcement_expgroup")

model_list_2 <- list(col_4_regression, col_5_regression, col_6_regression)

model_data <- model_list_2 %>%
  map_dfr(extract_7_2_coefs, .id = "model")

table_data_2 <- model_data %>%
  pivot_wider(names_from = model, values_from = values)

cbind(table_data_1, table_data_2[, -1])

feature,1,2,3,1,2,3
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
expgroup_treatment,,,,,,
announcement_expgroup,,,,,,


## Appendix A2

In [14]:
names(tc_comparison_data)

ERROR: Error in eval(expr, envir, enclos): object 'tc_comparison_data' not found


In [ ]:
# Load the tc_comparison_data
tc_comparison_data <- read_dta("data/tc_comparison.dta")

# Means by group
tc_comparison_data %>%
  group_by(expgroup) %>%
  summarise_all(mean, na.rm = TRUE)

# Overall SD
sapply(tc_comparison_data, sd, na.rm = TRUE)

# Significance of difference
cors <- cor(tc_comparison_data[ , c("expgroup", "age", "men", "second_technical", "high_school",
                      "tertiary_technical", "university", "prior_experience", "tenure", "married", "children",
                      "ageyoungestchild", "rental", "costofcommute", "internet", "bedroom",
                      "basewage", "bonus", "grosswage")])
corr.sig <- cor.mtest(cors, conf.level = 0.95)

# Significance of difference within type
tc_comparison_data$share <- tc_comparison_data$expgroup - 0.5
tc_comparison_data$type[tc_comparison_data$type == 3] <- 2
tc_comparison_data$type <- factor(tc_comparison_data$type)

model <- lm(share ~ type, data = tc_comparison_data)
print(coeftest(model, vcov = vcovHC(model, type = "HC0")))

expgroup,personid,prior_experience,age,tenure,basewage,bonus,grosswage,ageyoungestchild,costofcommute,⋯,rental,bedroom,men,married,second_technical,high_school,tertiary_technical,university,internet,type
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,28865.02,16.75330,24.34746,28.25424,1562.799,1092.587,3003.362,0.7118644,8.337827,⋯,0.2033898,0.9915254,0.4661017,0.3220339,0.4745763,0.1355932,0.3559322,0.03389831,1.0000000,1.906780
1,30299.21,18.96183,24.43511,26.13740,1539.864,1030.902,2949.730,0.5267176,7.891742,⋯,0.2442748,0.9694656,0.4656489,0.2213740,0.4580153,0.1755725,0.3511450,0.01526718,0.9923664,2.053435


personid   prior_experience                age             tenure 
      1.184494e+04       2.588153e+01       3.545459e+00       2.192174e+01 
          basewage              bonus          grosswage   ageyoungestchild 
      1.614480e+02       6.253335e+02       7.896269e+02       1.929177e+00 
     costofcommute           children             rental            bedroom 
      6.958090e+00       3.787410e-01       4.183571e-01       1.405576e-01 
          expgroup                men            married   second_technical 
      5.003238e-01       4.998380e-01       4.443729e-01       4.998380e-01 
       high_school tertiary_technical         university           internet 
      3.641803e-01       4.789926e-01       1.536573e-01       6.337243e-02 
              type 
      1.273071e+00


t test of coefficients:

             Estimate Std. Error t value Pr(>|t|)
(Intercept) 0.0074627  0.0431886  0.1728   0.8630
type2       0.0219491  0.0743573  0.2952   0.7681
type4       0.0203151  0.0937458  0.2167   0.8286
type5       0.2198100  0.1410561  1.5583   0.1204



In [ ]:
tc_comparison_data_ones = subset(tc_comparison_data, type == 1)
data %>%
  group_by(expgroup) %>%
  summarise_all(mean, na.rm = TRUE)

sapply(data, sd, na.rm = TRUE)

cors <- cor(data[ , c("expgroup", "age", "men", "second_technical", "high_school",
                      "tertiary_technical", "university", "prior_experience", "tenure", "married", "children",
                      "ageyoungestchild", "rental", "costofcommute", "internet", "bedroom",
                      "basewage", "bonus", "grosswage")])
corr.sig <- cor.mtest(cors, conf.level = 0.95)

expgroup,personid,prior_experience,age,tenure,basewage,bonus,grosswage,ageyoungestchild,costofcommute,⋯,rental,bedroom,men,married,second_technical,high_school,tertiary_technical,university,internet,type
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,28865.02,16.75330,24.34746,28.25424,1562.799,1092.587,3003.362,0.7118644,8.337827,⋯,0.2033898,0.9915254,0.4661017,0.3220339,0.4745763,0.1355932,0.3559322,0.03389831,1.0000000,1.906780
1,30299.21,18.96183,24.43511,26.13740,1539.864,1030.902,2949.730,0.5267176,7.891742,⋯,0.2442748,0.9694656,0.4656489,0.2213740,0.4580153,0.1755725,0.3511450,0.01526718,0.9923664,2.053435


personid   prior_experience                age             tenure 
      1.184494e+04       2.588153e+01       3.545459e+00       2.192174e+01 
          basewage              bonus          grosswage   ageyoungestchild 
      1.614480e+02       6.253335e+02       7.896269e+02       1.929177e+00 
     costofcommute           children             rental            bedroom 
      6.958090e+00       3.787410e-01       4.183571e-01       1.405576e-01 
          expgroup                men            married   second_technical 
      5.003238e-01       4.998380e-01       4.443729e-01       4.998380e-01 
       high_school tertiary_technical         university           internet 
      3.641803e-01       4.789926e-01       1.536573e-01       6.337243e-02 
              type 
      1.273071e+00

## Appendix O1

In [ ]:
data <- read_dta("data/summary_volunteer.dta")

A <- matrix(0, nrow = 18, ncol = 3)

j <- 1
variables <- c("children", "married", "commute", "bedroom", "high_educ", "tenure", "grosswage", "age", "men")

for (i in variables) {
  # volunteer == 1
  vol1 <- data[data$volunteer == 1, ][[i]]
  A[j, 1] <- mean(vol1, na.rm = TRUE)
  A[j + 1, 1] <- sd(vol1, na.rm = TRUE)
  
  # volunteer == 0
  vol0 <- data[data$volunteer == 0, ][[i]]
  A[j, 2] <- mean(vol0, na.rm = TRUE)
  A[j + 1, 2] <- sd(vol0, na.rm = TRUE)
  
  model <- lm(data[[i]] ~ data$volunteer)
  coefs <- coef(summary(model))
  A[j, 3] <- coefs[2, 3]  # t-statistic
  
  j <- j + 2
}

# Convert matrix to data frame
df <- as.data.frame(A)

# Add row labels
row_labels <- c()
for (i in seq_along(variables)) {
  row_labels <- c(row_labels, paste0(variables[i], " mean"), paste0(variables[i], " sd"))
}
rownames(df) <- row_labels

# Add column labels
colnames(df) <- c("volunteer", "non-volunteer", "t statistic")

# Print the data frame
df

,volunteer,non-volunteer,t statistic
,<dbl>,<dbl>,<dbl>
children mean,0.1033797,0.06517312,2.16812968
children sd,0.3047573,0.24708282,0.00000000
married mean,0.1749503,0.12627291,2.14633270
married sd,0.3803028,0.33249545,0.00000000
commute mean,86.4974155,74.46558045,3.37849725
commute sd,60.4671997,51.32059157,0.00000000
bedroom mean,0.6640159,0.54378819,3.90174405
bedroom sd,0.4728036,0.49858689,0.00000000
high_educ mean,0.3737575,0.46232179,-2.83944753


## Appendix O4

In [ ]:
# Load the treatment_effect_data
treatment_effect_data <- read_dta("data/treatment_effect.dta")
treatment_effect_data$personid <- as.character(treatment_effect_data$personid)

# List of variables
vars <- c("children", "women", "childrenwomen", "commute120", "rental", "young",
          "short_exper", "short_tenure", "livewparents", "livewspouse",
          "livewfriend", "perform10")

for (var in vars) {
  # Create interaction terms
  treatment_effect_data[, paste0("exper_treat_", var)] <- treatment_effect_data[, paste0("experiment_treatment")] * treatment_effect_data[, var]
  
  # Fit the regression model
  formula <- as.formula(paste0("perform1 ~ exper_treat_", var, " + experiment_treatment + ", var, " + factor(year_week)"))
  model <- lm(formula = formula,
              data = subset(treatment_effect_data, year_week != 201049))
  
  # Print the model summary with clustered standard errors
  print(coeftest(model, vcov = vcovCL, cluster = ~personid))
  cat("\n\n")
}


t test of coefficients:

                         Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)             -1.674355   0.056097 -29.8477 < 2.2e-16 ***
exper_treat_children    -0.124530   0.165913  -0.7506  0.452921    
experiment_treatment     0.238633   0.086826   2.7484  0.005995 ** 
children                 0.302966   0.095935   3.1580  0.001591 ** 
factor(year_week)201002  1.629406   0.083645  19.4800 < 2.2e-16 ***
factor(year_week)201003  1.771122   0.087699  20.1954 < 2.2e-16 ***
factor(year_week)201004  1.880553   0.088788  21.1802 < 2.2e-16 ***
factor(year_week)201005  1.758794   0.090121  19.5159 < 2.2e-16 ***
factor(year_week)201006  1.755440   0.084481  20.7790 < 2.2e-16 ***
factor(year_week)201007  1.507969   0.070596  21.3606 < 2.2e-16 ***
factor(year_week)201008  1.661564   0.091643  18.1308 < 2.2e-16 ***
factor(year_week)201009  1.613549   0.076587  21.0683 < 2.2e-16 ***
factor(year_week)201010  1.716357   0.078881  21.7588 < 2.2e-16 ***
factor(year_week)20101

## Appendix O5

In [ ]:
performance_during_exper_01 = subset(performance_during_exper, expgroup %in% c(0, 1))

model <- lm(perform1 ~ experiment_home + factor(year_week) + factor(personid),
            data = subset(performance_during_exper_01, year_week != 201049))
print(summary(model))
cat("\n\n")


Call:
lm(formula = perform1 ~ experiment_home + factor(year_week) + 
    factor(personid), data = subset(performance_during_exper_01, 
    year_week != 201049))

Residuals:
    Min      1Q  Median      3Q     Max 
-4.2678 -0.4178  0.0396  0.4693  3.8453 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)             -1.526483   0.116529 -13.100  < 2e-16 ***
experiment_home          0.274616   0.023234  11.820  < 2e-16 ***
factor(year_week)201002  1.629397   0.084175  19.357  < 2e-16 ***
factor(year_week)201003  1.769395   0.084278  20.995  < 2e-16 ***
factor(year_week)201004  1.874638   0.083969  22.325  < 2e-16 ***
factor(year_week)201005  1.777918   0.083575  21.273  < 2e-16 ***
factor(year_week)201006  1.766202   0.083574  21.133  < 2e-16 ***
factor(year_week)201007  1.534181   0.083381  18.400  < 2e-16 ***
factor(year_week)201008  1.678445   0.083288  20.152  < 2e-16 ***
factor(year_week)201009  1.627501   0.083107  19.583  < 2e-16 ***
fac